# Digital Assistant

In [ ]:
# - Frames

startFrame = {
    'done' : False,
    'nextState' : None
}

unclearFrame = {
    'task' : None,
    'done' : False,
    'nextState' : None
}

busFrame = {
    'fromLocation': None,
    'toLocation' : None,
    'time':None,
    'done' : False,
    'nextState' : None
} 

# bank start

bankFrame = {
    'done' : False,
    'nextState' : None
}

# bank insurance

insuranceFrame = {
    'customer' : None, # If the user is already a customer
    'ID number' : None, # What the users ID number is
    'done' : False, # If we are done and ready to move to another state
    'nextState' : None # What the next state should be
}

knownCustomerInsurance = {
    'upgrade' : False,
    'delay' : False,
    'cancel' : False,
    'done' : False,
    'nextState' : None
}

newCustomerInsurance = {
    'home' : False,
    'personal' : False,
    'done' : False,
    'nextState' : None
}

#bank loan

loanFrame = {
    'customer' : None,
    'ID number' : None,
    'done' : False,
    'nextState' : None
}

knownCustomerLoanFrame = {
    'delay' : False,
    'upgrade' : False,
    'done' : False,
    'nextState' : None
}

newCustomerLoanFrame = {
    'amount' : None,
    'done' : False,
    'nextState' : None
}

######## "api functions" / "database functions"

def getBusInfo(request):
    fromLocation = request.frame['fromLocation']
    toLocation = request.frame['toLocation']
    time = request.frame['time']
    print(f"Bus from {fromLocation} to {toLocation} leaves at {time}")

def knownInsurance(request):
    if request.frame['delay']:
        print('We have delayed your payment with one month')
    if request.frame['upgrade']:
        print('We have upgraded your insurance by 10%')

def newInsurance(request):
    idNr = request.prev.frame['ID number']
    if request.frame['home']:
        print(f'We have added a good home insurance for {idNr}')
    if request.frame['personal']:
        print(f'We have added a good personal insurance for {idNr}')

def knownLoan(request):
    if request.frame['delay']:
        print('We have delayed your payment with one month')
    if request.frame['upgrade']:
        print('We have upgraded your loan by 10%')

def newLoan(request):
    idNr = request.prev.frame['ID number']
    amount = request.frame['amount']
    print(f'We have sent a loan to {idNr} of {amount}')

#### Pointer to all the frames used by the NLU

frames = {
    'bus' : busFrame,
    'bank' : bankFrame,
    'insurance' : insuranceFrame,
    'knownInsurance' : knownCustomerInsurance,
    'newInsurance' : newCustomerInsurance,
    'loan' : loanFrame,
    'knownLoan' : knownCustomerLoanFrame,
    'newLoan' : newCustomerLoanFrame,
    'start' : startFrame,
    'unclear' : unclearFrame
}

##### Pointers to API / database functions used by the taskmanager

request_dict = {
    'bus' : getBusInfo,
    'knownInsurance' : knownInsurance,
    'newInsurance' : newInsurance,
    'knownLoan' : knownLoan,
    'newLoan' : newLoan
}

###### Word list of different choices to fill the frames with and synonyms, used by the NLU

requests = [
    ['bus','busride','bus ride','public transport','transport','transportation','ride'],
    ['bank','banking','Bank','Banking'],
    ['loan','credit','mortage','lending'],
    ['insurance','security','protection','Insurance'],
    ['unclear']
]

locations = [
    ['from stockholm','from sthlm','from Stockholm','From stockholm','From sthlm','From Stockholm'],
    ['from gothenburg','from gbg','from Goteborg','from Gothenburg','from goteborg','From gothenburg','From gbg','From Goteborg','From Gothenburg','From goteborg'],
    ['from malmo','from Malmo','From malmo','From Malmo'],
    ['stockholm','sthlm','Stockholm'],
    ['gothenburg','gbg','Goteborg','Gothenburg','goteborg'],
    ['malmo','Malmo']
]

insuranceKnownTerms = [
    ['upgrade','improve','reform','enhance','cheap','cheaper','affordable'],
    ['delay','broke',"can't",'this month','right now'],
    ['cancel',"don't want",'stop']
]

insuranceNewTerms = [
    ['personal','individual'],
    ['home','house']
]

loanKnownTerms = [
    ['upgrade','improve','reform','enhance','cheap','cheaper','affordable'],
    ['delay','broke',"can't",'this month','right now']
]

times = ['00:00','01:00','02:00','03:00','04:00','05:00','06:00','07:00','08:00','09:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00','17:00','18:00','19:00','20:00','21:00','22:00','23:00']


In [ ]:
class Request:
    
    def __init__(self, request = 'start', prev = None):
        self.req_type = request
        self.frame = frames[request].copy()
        self.nextState = None # will be either getInfo or name of a new frame.
        self.prev = prev
    
    def getPrev(self):
        return self.prev

    def addPrev(self, request):
        self.prev = request


In [ ]:
class TaskManager:
    # input is complete frame 
    # Should retrun search result from database, but since were not extending any model past the first "database" search this is not implemented.
    def get_info(self, request):
        funct = request_dict[request.req_type] #
        res = funct(request) # result : 



In [ ]:
# A few parts of this code could be extracted to helper functions and be reused in multiple functions. This would help with readability and 
# would make the class more easily extendable.

class NLU:
    def __init__(self):
        pass 
    
    def parse_input(self,input_string, request):
        # checkChangeMind(request) maybe?
        if 'exit' in input_string:
            return 0
        if request.req_type == 'start':
            return self.startRequest(input_string, request)
        if request.req_type == 'bus':
            return self.busRequest(input_string, request)
        if request.req_type == 'bank':
            return self.bankRequest(input_string, request)
        if request.req_type == 'insurance':
            return self.insuranceRequest(input_string, request)
        if request.req_type == 'knownInsurance':
            return self.knownInsuranceRequest(input_string, request)
        if request.req_type == 'newInsurance':
            return self.newInsuranceRequest(input_string, request)
        if request.req_type == 'loan':
            return self.loanRequest(input_string, request)
        if request.req_type == 'knownLoan':
            return self.knownLoanRequest(input_string, request)
        if request.req_type == 'newLoan':
            return self.newLoanRequest(input_string, request)
        if request.req_type == 'unclear':
            return self.unclearRequest(input_string, request)
    
    def startRequest(self, input_string, request):
        for reqElems in requests:
            for reqElem in reqElems:
                if reqElem in input_string:
                    request.nextState = reqElems[0]
                    request.frame['done'] = True
        if request.frame['done'] == False:
                print("sorry I did not understand that")
    
    def bankRequest(self, input_string, request):
        self.startRequest(input_string, request)
    
    def insuranceRequest(self, input_string, request):
        if request.frame['customer'] is None:
            if "yes" in input_string:
                request.frame['customer'] = True
                return
            if "no" in input_string:
                request.frame['customer'] = False
                return
        if request.frame['ID number'] is None:
            # Should be check format for ID number and extract here, but it's not added
            request.frame['ID number'] = input_string
            request.frame['done'] = True
            if request.frame['customer']:
                request.nextState = 'knownInsurance'
            else:
                request.nextState = 'newInsurance'
        if request.frame['done'] == False:
            print("sorry I did not understand that")

    def knownInsuranceRequest(self, input_string, request):
        for terms in insuranceKnownTerms:
            for term in terms:
                if term in input_string:
                    request.frame[terms[0]] = True
                    request.frame['done'] = True
                    request.nextState = 'getInfo'
                    return
        print("Sorry I did not understand that")

    
    def newInsuranceRequest(self, input_string, request):
        for terms in insuranceNewTerms:
            for term in terms:
                if term in input_string:
                    request.frame[terms[0]] = True
                    request.frame['done'] = True
                    request.nextState = 'getInfo'
                    print(f"Let me find a good {terms[0]} insurance for you...")
                    return
        print("Sorry I did not understand that")

    
    def loanRequest(self, input_string, request):
        if request.frame['customer'] is None:
            if "yes" in input_string:
                request.frame['customer'] = True
                return
            if "no" in input_string:
                request.frame['customer'] = False
                return
        if request.frame['ID number'] is None:
            # Should be check format for ID number and extract here, but it's not added
            request.frame['ID number'] = input_string
            request.frame['done'] = True
            if request.frame['customer']:
                request.nextState = 'knownLoan'
            else:
                request.nextState = 'newLoan'
        if request.frame['done'] == False:
            print("sorry I did not understand that")
    
    def knownLoanRequest(self, input_string, request):
        for terms in loanKnownTerms:
            for term in terms:
                if term in input_string:
                    request.frame[terms[0]] = True
                    request.frame['done'] = True
                    request.nextState = 'getInfo'
                    return
        print("Sorry I did not understand that")

    
    def newLoanRequest(self, input_string, request):
        # Here we should check for format and extract the number etcetc, but it's not added.
        request.frame['amount'] = input_string
        request.nextState = 'getInfo'
        request.frame['done'] = True
        print("Let me find a good loan for you...")
    
    def busRequest(self, input_string, request):
        if request.frame['fromLocation'] == None:
            stop = False
            for locationVars in locations: # locations has lists of words that mean the same thing
                if stop:
                    break
                for location in locationVars: # loop each word that mean the same thing
                    if location in input_string:
                        request.frame['fromLocation'] = locationVars[0] # first word in list represents formal meaning of word
                        input_string = input_string.replace(location,'') # remove location from string
                        stop = True
                        break
            if request.frame['fromLocation'] == None: # if not filled print msg
                print("sorry I did not understand that")
                return
            if "to" in input_string: 
                self.busRequest(input_string,request) # if to in message call busrequest again
            return
        if request.frame['toLocation'] == None:
            for locationVars in locations:
                for location in locationVars:
                    if location in input_string:
                        request.frame['toLocation'] = locationVars[0]
            if request.frame['toLocation'] == None:
                print("sorry I did not understand that")
            return
        if request.frame['time'] == None:
            for time in times:
                if time in input_string:
                    request.frame['time'] = time
                    request.frame['done'] = True
                    request.nextState = 'getInfo'
                    print(f"Do you want to travel from {request.frame['fromLocation']} to {request.frame['toLocation']} at {request.frame['time']}? yes/no")
                    x = input()
                    if "yes" in x:
                        return
                    if "no" in x:
                        request.frame = unclearFrame.copy() # not added..
                        return
            if request.frame['done'] == False:
                print("sorry I did not understand that")
            return

In [ ]:
class NLG:
    x=0
    def __init__(self):
        pass
    
    def generate(self, request): # find appropriate function call based on request we're working on
        if request.req_type == 'start':
            return self.startRequest(request)
        if request.req_type == 'bus':
            return self.busRequest(request)
        if request.req_type == 'bank':
            return self.bankRequest(request)
        if request.req_type == 'insurance':
            return self.insuranceRequest(request)
        if request.req_type == 'knownInsurance':
            return self.knownInsuranceRequest(request)
        if request.req_type == 'newInsurance':
            return self.newInsuranceRequest(request)
        if request.req_type == 'loan':
            return self.loanRequest(request)
        if request.req_type == 'knownLoan':
            return self.knownLoanRequest(request)
        if request.req_type == 'newLoan':
            return self.newLoanRequest(request)
        if request.req_type == 'unclear':
            return self.unclearRequest(request)
        
    def bankRequest(self,request):
        print('What bank thing can i Assist you with?')
        x = input()
        print("you: " + x)
        return x
    
    def insuranceRequest(self,request):
        if request.frame['customer'] == None:
            print("Thanks for your interest in our insurances.")
            print('Are you already a customer here?')
            x = input()
            print("you: " + x)
            return x
        if request.frame['ID number'] == None:
            print('What is your ID number?')
            x = input()
            print("you: " + x)
            return x
    
    def knownInsuranceRequest(self,request):
        print('What do you want to do with your insurance?')
        x = input()
        print("you: " + x)
        return x
    
    def newInsuranceRequest(self,request):
        print('What kind of insurance are you interested in?')
        x = input()
        print("you: " + x)
        return x
    
    def loanRequest(self,request):
        if request.frame['customer'] == None:
            print("Thanks for your interest in our loans.")
            print('Are you already a customer here?')
            x = input()
            print("you: " + x)
            return x
        if request.frame['ID number'] == None:
            print('What is your ID number?')
            x = input()
            print("you: " + x)
            return x
    
    def knownLoanRequest(self,request):
        print('What do you want to do with your loan?')
        x = input()
        print("you: " + x)
        return x
    
    def newLoanRequest(self,request):
        print('How much money do you want to lend?')
        x = input()
        print("you: " + x)
        return x

    def startRequest(self, request):
        print("Hi, what can I assist you with?")
        x = input()
        print("you: " + x)
        return x
    
    def busRequest(self, request):
        if request.frame['fromLocation'] == None:
            print('From what location do you want to take the bus?')
            x = input()
            print("you: " + x)
            return x
        if request.frame['toLocation'] == None:
            print('Where do you want to travel to?')
            x = input()
            print("you: " + x)
            return x
        if request.frame['time'] == None:
            print('What time do you want to travel? Express time in full hour form e.g.: 21:00')
            x = input()
            print("you: " + x)
            return x
    
    def unclearRequest(self,request):
        print('Spam')

In [ ]:
class DialogueManager:

    def __init__(self):
        self.request = Request('start')
        self.nlu = NLU()
        self.nlg = NLG()
        self.taskm = TaskManager()
    
    def start(self):
        while(True):
            x = self.nlg.generate(self.request)
            self.nlu.parse_input(x, self.request)
            if self.request.nextState == 'getInfo':
                self.taskm.get_info(self.request)
                return
            if not (self.request.nextState is None):
                newRequest = Request(self.request.nextState)
                newRequest.addPrev(self.request)
                self.request = newRequest

In [ ]:
asd = DialogueManager()
asd.start()

Hi, what can I assist you with?
you: I want to take the bus
From what location do you want to take the bus?
you: Stockholm to Gothenburg
What time do you want to travel? Express time in full hour form e.g.: 21:00
you: 12:00
Do you want to travel from stockholm to gothenburg at 12:00? yes/no
Bus from stockholm to gothenburg leaves at 12:00


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=24258edb-8229-4534-80c4-059c7fbaa123' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>